In [1]:
import polars as pl
import fastf1 as ff1
import pandas as pd

In [2]:
race_data = pl.scan_parquet("./static/data/race_data.parquet").select(["x","y","year","event_name","driver_number","artifical_pos"])

In [3]:
all_gps = []
for year in range(2020,2025):
    temp_df = pd.DataFrame(ff1.get_event_schedule(year)["EventName"])
    temp_df["year"] = year
    temp_df["round_number"] = pd.DataFrame(ff1.get_event_schedule(year)["RoundNumber"])
    if year != 2024:
        all_gps.append(temp_df)
    else:
        all_gps.append(temp_df.loc[:8])




df_all_gps = pd.concat(all_gps)
lf_all_gps = pl.LazyFrame(df_all_gps)

req         WARNING 	DEFAULT CACHE ENABLED! (3.57 GB) /Users/max/Library/Caches/fastf1


In [4]:
df_all_gps

,EventName,year,round_number
0,Pre-Season Test 1,2020,0
1,Pre-Season Test 2,2020,0
2,Austrian Grand Prix,2020,1
3,Styrian Grand Prix,2020,2
4,Hungarian Grand Prix,2020,3
...,...,...,...
4,Japanese Grand Prix,2024,4
5,Chinese Grand Prix,2024,5
6,Miami Grand Prix,2024,6
7,Emilia Romagna Grand Prix,2024,7


In [5]:
race_data = race_data.join(other=lf_all_gps,left_on=["year","event_name"], right_on=["year","EventName"])
race_data = race_data.select(["x","y","year","round_number","driver_number","artifical_pos"])

In [6]:
race_data = race_data.collect()

In [13]:
race_data = race_data.with_columns(
    race_data.group_by(['driver_number', 'year',"round_number"]).agg(pl.col("x").cum_count().alias("pos_index"))
)

ShapeError: unable to add a column of length 1794 to a DataFrame of height 44959521

In [15]:
session = ff1.get_session(2024,1,"R")
session.load()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.7]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [16]:
session.laps.pick_driver("1").get_pos_data()

,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2024-03-02 15:03:42.460,OnTrack,-280,3550,-157,pos,0 days 00:00:00.118000,0 days 01:00:00.029000
1,2024-03-02 15:03:42.700,OnTrack,-280,3550,-157,pos,0 days 00:00:00.358000,0 days 01:00:00.269000
2,2024-03-02 15:03:43.040,OnTrack,-280,3558,-157,pos,0 days 00:00:00.698000,0 days 01:00:00.609000
3,2024-03-02 15:03:43.460,OnTrack,-280,3574,-157,pos,0 days 00:00:01.118000,0 days 01:00:01.029000
4,2024-03-02 15:03:43.619,OnTrack,-279,3585,-157,pos,0 days 00:00:01.277000,0 days 01:00:01.188000
...,...,...,...,...,...,...,...,...
21238,2024-03-02 16:35:26.199,OnTrack,-423,436,-159,pos,0 days 01:31:43.857000,0 days 02:31:43.768000
21239,2024-03-02 16:35:26.478,OnTrack,-415,572,-159,pos,0 days 01:31:44.136000,0 days 02:31:44.047000
21240,2024-03-02 16:35:26.778,OnTrack,-404,795,-159,pos,0 days 01:31:44.436000,0 days 02:31:44.347000
21241,2024-03-02 16:35:26.938,OnTrack,-398,917,-159,pos,0 days 01:31:44.596000,0 days 02:31:44.507000
